<a href="https://colab.research.google.com/github/erictong0/FrederickLab2023/blob/main/Freezing%20Time/Older_Euler's_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#I don't think this is necessary but it's standard
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt

#step size in nanoseconds
stepSizeInput = .5

#radius in microns
radiusInput = 40

#Temperatures in K
startingTemperatureInput = 293
endingTemperatureInput = 78 #can't be below 77K, as that's the the temp of LN2

#percent glycerol by volume
percentGlycerolInput = .15

#densities
glycerolDensity = 1261 #kg/m^3 @20C
waterDensity = 998.23 #kg/m^3 @20C

#specific heat
specificHeatInput = 2 #cal/g/C

###    end of changable variables    ###
#making all conversions to standard units
r = radiusInput * 10**-6
step = stepSizeInput * 10**-9
temp = startingTemperatureInput
eTemp = endingTemperatureInput

#calculating other variables
startingVolume = 4/3 * np.pi * r**3
glycerolVolume = startingVolume * percentGlycerolInput
waterVolume = startingVolume - glycerolVolume
glycerolMass = glycerolVolume * glycerolDensity
waterMass = waterVolume * waterDensity
totalMass = glycerolMass + waterMass
print(totalMass)
#constant
c = 5491.82394537

#comes from http://www.met.reading.ac.uk/~sws04cdw/viscosity_calc.html
def calcDensity(temp):

  T = temp - 273

  glycerolDen = (1273.3-0.6121*T)			#Density of Glycerol (kg/m3)
  if T > 0:
    waterDen = 1000 	#Density of water (kg/m3)
  else:
    waterDen = 940
  #(1-math.pow(((abs(T-4))/622),1.7))
  waterVol = waterMass / waterDen
  glycerolVol = glycerolMass / glycerolDen

  totalMass=glycerolMass+waterMass
  mass_fraction=glycerolMass/totalMass

  vol_fraction= glycerolVol/(glycerolVol+waterVol)

  ##Andreas Volk polynomial method
  contraction_av = 1-math.pow(3.520E-8*((mass_fraction*100)),3)+math.pow(1.027E-6*((mass_fraction*100)),2)+2.5E-4*(mass_fraction*100)-1.691E-4
  contraction = 1+contraction_av/100
  #print(glycerolDen)
  density_mix=(glycerolDen*vol_fraction+waterDen*(1-vol_fraction))*contraction

  return density_mix

"""  mass_glycerol = glycerolMass
  mass_water = waterMass
  total_mass=mass_glycerol+mass_water
  mass_fraction=mass_glycerol/total_mass

  density_glycerol=1273.3-0.6121*temp
  density_water=1000*(1-((abs(temp-3.98))/615)**1.71)

  volume_glycerol=mass_glycerol/density_glycerol
  volume_water=mass_water/density_water
  total_volume=volume_glycerol+volume_water
  fraction_glyc=volume_glycerol/total_volume

  c=1.78E-6*temp**2-1.82E-4*temp+1.41E-2;
  contraction=1+(c*np.sin((mass_fraction)**1.31*np.pi)**0.81);

  print(contraction)
  density_mix=(density_glycerol*fraction_glyc+density_water*(1-fraction_glyc))*contraction"""


time = 0
tempTemp = temp-1
x = []
y = []

#Continues to calculate while the temperature is not met
while temp > (endingTemperatureInput):

  density = calcDensity(temp) #run the density through the formula

  changeInEnergy = np.sqrt((4 * np.pi * totalMass)/(3 * density)) * c * (temp - 77) #Use the formula
  specificHeat = specificHeatInput * 4200 #convert cal/g/C to J/kg/K
  changeInTemp = changeInEnergy / totalMass / specificHeat #use specific heat + energy + mass to get change in temp
  #print(changeInTemp)
  temp -= changeInTemp * step #subtract because the change is from inside to outside
  if temp < tempTemp:
      x.append(time * 1E9)
      y.append(temp)
      tempTemp = temp - 1
      #print(str(round(time * 10E9)) + ": " + str(temp) + "K")
  time += step #add time

plt.xlabel("Time (nanoseconds)")
plt.ylabel("Temp (K)")
plt.title("Predicted temperature of " + str(radiusInput*2) + "um (diameter) particle over time")
plt.plot(x, y)
print(str(round(time * 1E9)) + " nanoseconds")

